In [8]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
from warnings import filterwarnings
import warnings
warnings.filterwarnings('ignore')

# preprocessing and cleaning
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from kaggle.api.kaggle_api_extended import KaggleApi

In [9]:
import os
from dotenv import load_dotenv

# Load biến môi trường từ file .env (file .env cần nằm trong cùng thư mục với notebook)
load_dotenv()


True

In [10]:
username=os.getenv('KAGGLE_USERNAME')
key=os.getenv('KAGGLE_KEY')



# <div style="padding: 30px; color:white; margin:10; font-size:75%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b><span style='color:#F1A424'>1 |</span></b> <b> LOAD DATA</b></div>

In [11]:
api = KaggleApi()
api.authenticate()

# Tải TRỰC TIẾP về path bạn muốn
target_path = "/Users/minhhieu/Documents/AI_VN"
api.dataset_download_files(
    "dongrelaxman/amazon-reviews-dataset",
    path=target_path,
    unzip=True
)
print(f"✅ Dataset đã được tải về: {target_path}")

Dataset URL: https://www.kaggle.com/datasets/dongrelaxman/amazon-reviews-dataset
✅ Dataset đã được tải về: /Users/minhhieu/Documents/AI_VN


In [29]:
import pandas as pd

# Thử với engine 'python' thay vì 'c' (mặc định)
df = pd.read_csv(
    '/Users/minhhieu/Documents/AI_VN/Amazon_Reviews.csv',
    engine='python',
    on_bad_lines='skip'  # Bỏ qua các dòng lỗi
)

In [30]:
df.shape

(21214, 9)

In [31]:
df.head()

,Reviewer Name,Profile Link,Country,Review Count,Review Date,Rating,Review Title,Review Text,Date of Experience
0,Eugene ath,/users/66e8185ff1598352d6b3701a,US,1 review,2024-09-16T13:44:26.000Z,Rated 1 out of 5 stars,A Store That Doesn't Want to Sell Anything,"I registered on the website, tried to order a ...","September 16, 2024"
1,Daniel ohalloran,/users/5d75e460200c1f6a6373648c,GB,9 reviews,2024-09-16T18:26:46.000Z,Rated 1 out of 5 stars,Had multiple orders one turned up and…,Had multiple orders one turned up and driver h...,"September 16, 2024"
2,p fisher,/users/546cfcf1000064000197b88f,GB,90 reviews,2024-09-16T21:47:39.000Z,Rated 1 out of 5 stars,I informed these reprobates,I informed these reprobates that I WOULD NOT B...,"September 16, 2024"
3,Greg Dunn,/users/62c35cdbacc0ea0012ccaffa,AU,5 reviews,2024-09-17T07:15:49.000Z,Rated 1 out of 5 stars,Advertise one price then increase it on website,I have bought from Amazon before and no proble...,"September 17, 2024"
4,Sheila Hannah,/users/5ddbe429478d88251550610e,GB,8 reviews,2024-09-16T18:37:17.000Z,Rated 1 out of 5 stars,If I could give a lower rate I would,If I could give a lower rate I would! I cancel...,"September 16, 2024"


# <div style="padding: 30px; color:white; margin:10; font-size:75%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b><span style='color:#F1A424'>2 |</span></b> <b> Preprocessing</b></div>
 <!-- - missing values
 - duplicates
 - Normalization the location -->

In [32]:
df.isnull().sum()

Reviewer Name           0
Profile Link           51
Country               160
Review Count          159
Review Date           159
Rating                159
Review Title          159
Review Text           159
Date of Experience    267
dtype: int64

### Mapping Country 

In [33]:
print(df["Country"].unique())

['US' 'GB' 'AU' 'JP' 'CA' 'ZA' 'IN' 'BE' 'AE' 'NZ' 'IL' 'NL' 'SE' 'DK'
 'JM' 'DE' 'MY' 'FR' 'ES' 'IT' 'PR' 'KW' 'PK' 'PT' 'TW' 'IE' 'DO' 'TR'
 'HK' 'ME' 'FI' 'CH' 'CO' 'CY' 'PA' 'ID' 'EG' 'HR' 'BR' 'GR' 'PL' 'NI'
 'TH' 'SK' 'CN' 'VG' 'BH' 'CZ' 'GG' 'AT' 'MA' 'CV' 'GE' 'LT' 'MT' 'MC'
 'RU' 'UA' 'AG' 'OM' 'VI' 'VN' 'KH' 'NG' 'QA' 'NO' 'GH' 'CR' 'BO' 'MQ'
 'KE' 'PE' 'BS' 'RO' 'JE' 'PH' 'DZ' 'RS' 'AZ' 'AF' 'SA' 'AM' 'MV' 'EE'
 'HU' 'CM' 'PG' 'AR' 'BG' 'MX' 'LV' 'SL' 'CD' 'MO' 'GI' 'LU' 'AO' 'BD'
 'KR' 'KZ' 'ET' 'FJ' 'SG' 'UG' 'SR' 'EC' 'BY' 'SI' 'TN' 'HN' 'VE' 'NP'
 'BB' 'CW' 'GF' 'BZ' 'CL' 'GT' 'KG' 'MG' 'IM' 'TT' 'PY' 'LK' 'BA' nan 'LA'
 'TZ' 'IQ' 'BQ' 'GY' 'MN' 'IS' 'MD' 'UY' 'SO' 'RW' 'MU' 'BM' 'LB' 'IR'
 'JO' 'SV' 'BW' 'AD' 'CI' 'ZM' 'MK' None 'MM']


In [34]:
country_map = {
    'US': 'United States',
    'GB': 'United Kingdom',
    'AU': 'Australia',
    'JP': 'Japan',
    'CA': 'Canada',
    'ZA': 'South Africa',
    'IN': 'India',
    'BE': 'Belgium',
    'AE': 'United Arab Emirates',
    'NZ': 'New Zealand',
    'IL': 'Israel',
    'NL': 'Netherlands',
    'SE': 'Sweden',
    'DK': 'Denmark',
    'JM': 'Jamaica',
    'DE': 'Germany',
    'MY': 'Malaysia',
    'FR': 'France',
    'ES': 'Spain',
    'IT': 'Italy',
    'PR': 'Puerto Rico',
    'KW': 'Kuwait',
    'PK': 'Pakistan',
    'PT': 'Portugal',
    'TW': 'Taiwan',
    'IE': 'Ireland',
    'DO': 'Dominican Republic',
    'TR': 'Turkey',
    'HK': 'Hong Kong',
    'ME': 'Montenegro',
    'FI': 'Finland',
    'CH': 'Switzerland',
    'CO': 'Colombia',
    'CY': 'Cyprus',
    'PA': 'Panama',
    'ID': 'Indonesia',
    'EG': 'Egypt',
    'HR': 'Croatia',
    'BR': 'Brazil',
    'GR': 'Greece',
    'PL': 'Poland',
    'NI': 'Nicaragua',
    'TH': 'Thailand',
    'SK': 'Slovakia',
    'CN': 'China',
    'VG': 'British Virgin Islands',
    'BH': 'Bahrain',
    'CZ': 'Czech Republic',
    'GG': 'Guernsey',
    'AT': 'Austria',
    'MA': 'Morocco',
    'CV': 'Cape Verde',
    'GE': 'Georgia',
    'LT': 'Lithuania',
    'MT': 'Malta',
    'MC': 'Monaco',
    'RU': 'Russia',
    'UA': 'Ukraine',
    'AG': 'Antigua and Barbuda',
    'OM': 'Oman',
    'VI': 'U.S. Virgin Islands',
    'VN': 'Vietnam',
    'KH': 'Cambodia',
    'NG': 'Nigeria',
    'QA': 'Qatar',
    'NO': 'Norway',
    'GH': 'Ghana',
    'CR': 'Costa Rica',
    'BO': 'Bolivia',
    'MQ': 'Martinique',
    'KE': 'Kenya',
    'PE': 'Peru',
    'BS': 'Bahamas',
    'RO': 'Romania',
    'JE': 'Jersey',
    'PH': 'Philippines',
    'DZ': 'Algeria',
    'RS': 'Serbia',
    'AZ': 'Azerbaijan',
    'AF': 'Afghanistan',
    'SA': 'Saudi Arabia',
    'AM': 'Armenia',
    'MV': 'Maldives',
    'EE': 'Estonia',
    'HU': 'Hungary',
    'CM': 'Cameroon',
    'PG': 'Papua New Guinea',
    'AR': 'Argentina',
    'BG': 'Bulgaria',
    'MX': 'Mexico',
    'LV': 'Latvia',
    'SL': 'Sierra Leone',
    'CD': 'Congo (DRC)',
    'MO': 'Macau',
    'GI': 'Gibraltar',
    'LU': 'Luxembourg',
    'AO': 'Angola',
    'BD': 'Bangladesh',
    'KR': 'South Korea',
    'KZ': 'Kazakhstan',
    'ET': 'Ethiopia',
    'FJ': 'Fiji',
    'SG': 'Singapore',
    'UG': 'Uganda',
    'SR': 'Suriname',
    'EC': 'Ecuador',
    'BY': 'Belarus',
    'SI': 'Slovenia',
    'TN': 'Tunisia',
    'HN': 'Honduras',
    'VE': 'Venezuela',
    'NP': 'Nepal',
    'BB': 'Barbados',
    'CW': 'Curacao',
    'GF': 'French Guiana',
    'BZ': 'Belize',
    'CL': 'Chile',
    'GT': 'Guatemala',
    'KG': 'Kyrgyzstan',
    'MG': 'Madagascar',
    'IM': 'Isle of Man',
    'TT': 'Trinidad and Tobago',
    'PY': 'Paraguay',
    'LK': 'Sri Lanka',
    'BA': 'Bosnia and Herzegovina',
    'LA': 'Laos',
    'TZ': 'Tanzania',
    'IQ': 'Iraq',
    'BQ': 'Bonaire, Saint Eustatius and Saba',
    'GY': 'Guyana',
    'MN': 'Mongolia',
    'IS': 'Iceland',
    'MD': 'Moldova',
    'UY': 'Uruguay',
    'SO': 'Somalia',
    'RW': 'Rwanda',
    'MU': 'Mauritius',
    'BM': 'Bermuda',
    'LB': 'Lebanon',
    'IR': 'Iran',
    'JO': 'Jordan',
    'SV': 'El Salvador',
    'BW': 'Botswana',
    'AD': 'Andorra',
    'CI': "Côte d'Ivoire",
    'ZM': 'Zambia',
    'MK': 'North Macedonia',
    'MM': 'Myanmar'
}


In [35]:
def convert_country(code):
    if pd.isna(code) or code is None or str(code).strip().lower() in ["", "nan", "none"]:
        return "Do not mention"
    
    code = str(code).strip().upper()
    
    return country_map.get(code, "Do not mention")

## Apply for cCountry column
df["Country"] = df["Country"].apply(convert_country)


In [36]:
print(df["Country"].unique())

['United States' 'United Kingdom' 'Australia' 'Japan' 'Canada'
 'South Africa' 'India' 'Belgium' 'United Arab Emirates' 'New Zealand'
 'Israel' 'Netherlands' 'Sweden' 'Denmark' 'Jamaica' 'Germany' 'Malaysia'
 'France' 'Spain' 'Italy' 'Puerto Rico' 'Kuwait' 'Pakistan' 'Portugal'
 'Taiwan' 'Ireland' 'Dominican Republic' 'Turkey' 'Hong Kong' 'Montenegro'
 'Finland' 'Switzerland' 'Colombia' 'Cyprus' 'Panama' 'Indonesia' 'Egypt'
 'Croatia' 'Brazil' 'Greece' 'Poland' 'Nicaragua' 'Thailand' 'Slovakia'
 'China' 'British Virgin Islands' 'Bahrain' 'Czech Republic' 'Guernsey'
 'Austria' 'Morocco' 'Cape Verde' 'Georgia' 'Lithuania' 'Malta' 'Monaco'
 'Russia' 'Ukraine' 'Antigua and Barbuda' 'Oman' 'U.S. Virgin Islands'
 'Vietnam' 'Cambodia' 'Nigeria' 'Qatar' 'Norway' 'Ghana' 'Costa Rica'
 'Bolivia' 'Martinique' 'Kenya' 'Peru' 'Bahamas' 'Romania' 'Jersey'
 'Philippines' 'Algeria' 'Serbia' 'Azerbaijan' 'Afghanistan'
 'Saudi Arabia' 'Armenia' 'Maldives' 'Estonia' 'Hungary' 'Cameroon'
 'Papua New Guine

In [37]:
df.isnull().sum()

Reviewer Name           0
Profile Link           51
Country                 0
Review Count          159
Review Date           159
Rating                159
Review Title          159
Review Text           159
Date of Experience    267
dtype: int64

In [38]:
df = df.drop(columns=["Profile Link"])


In [39]:
df.isnull().sum()

Reviewer Name           0
Country                 0
Review Count          159
Review Date           159
Rating                159
Review Title          159
Review Text           159
Date of Experience    267
dtype: int64

### Drop Review Title and Text

In [ ]:
missing_title_only = df[df["Review Title"].isna() & df["Review Text"].notna()]
print(len(missing_title_only))
# => mọi record mà review text null thì review title đều null ( drop được vì ảnh hưởng ít , có 159 records thôi )
# missing_text_only = df[df["Review Text"].isna() & df["Review Title"].notna()]
# print(len(missing_text_only))


0


In [42]:
df = df.dropna(subset=["Review Title", "Review Text"])


In [43]:
df.shape

(21055, 8)

In [44]:
df.isnull().sum()

Reviewer Name           0
Country                 0
Review Count            0
Review Date             0
Rating                  0
Review Title            0
Review Text             0
Date of Experience    108
dtype: int64

In [45]:
df["Date of Experience"] = df["Date of Experience"].fillna(df["Review Date"])


In [46]:
df.isnull().sum()

Reviewer Name         0
Country               0
Review Count          0
Review Date           0
Rating                0
Review Title          0
Review Text           0
Date of Experience    0
dtype: int64